In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

In [2]:
from utils import *
from datetime import date

In [3]:
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


### Batchnorm Training
- `model_bn` is the model with batchnorm layers
- train this model on the CIFAR 10 dataset

In [4]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.ReLU(),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.ReLU(),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.BatchNorm1d(1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [5]:
loss_history_bn = []
writer_bn = SummaryWriter(log_dir='runs/bn_stats-relu-%s' % date.today(), flush_secs=10)

In [6]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')
def criterion(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) #+ (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))


In [7]:
loss_tanh = train_model(model_bn, trainloader, criterion, optimizer = optim.Adam(list(model_bn.parameters()), lr=3e-4), epochs=20, writer=writer_bn)

[1,   300] loss: 84.639
[1,   600] loss: 83.800
[2,   300] loss: 67.353
[2,   600] loss: 48.714
[3,   300] loss: 53.240
[3,   600] loss: 72.234
[4,   300] loss: 42.358
[4,   600] loss: 41.034
[5,   300] loss: 23.588
[5,   600] loss: 45.450
[6,   300] loss: 44.843
[6,   600] loss: 37.670
[7,   300] loss: 20.766
[7,   600] loss: 39.614
[8,   300] loss: 23.454
[8,   600] loss: 25.141
[9,   300] loss: 11.887
[9,   600] loss: 15.695
[10,   300] loss: 13.196
[10,   600] loss: 28.277
[11,   300] loss: 10.491
[11,   600] loss: 7.982
[12,   300] loss: 6.155
[12,   600] loss: 13.527
[13,   300] loss: 7.490
[13,   600] loss: 7.707
[14,   300] loss: 11.113
[14,   600] loss: 6.812
[15,   300] loss: 8.155
[15,   600] loss: 12.976
[16,   300] loss: 14.308
[16,   600] loss: 4.191
[17,   300] loss: 15.237
[17,   600] loss: 2.085
[18,   300] loss: 3.824
[18,   600] loss: 5.433
[19,   300] loss: 3.479
[19,   600] loss: 11.724
[20,   300] loss: 5.210
[20,   600] loss: 8.366


In [8]:
stats_bn = print_stats(model_bn)

Mean:  0.028904855
Standard Deviation:  0.028047554
Sparsity:  0.0024671131325638344


In [9]:
bn_model_err = get_model_error(model_bn, testloader)

test error: 0.291
